In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install scikit-plot

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster

from sklearn import metrics 
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

import scikitplot as skplt


In [6]:
df = pd.read_csv("/content/drive/MyDrive/Data/OQ - Question Data - Part 1.csv")  
df = df.iloc[3:]
df.head(3)

,Quesh,Quesh Date,Total Respo,Avg Rating
3,Which one condiment could you not live without?,2022-01-07,18,3.78
4,"If you had to win a game to save your life, wh...",2022-01-06,18,3.67
5,What’s your favorite food combo?,2022-01-05,16,3.56


In [7]:
df2 = pd.read_csv("/content/drive/MyDrive/Data/sentiment_score.csv")
df2.head(3)

,Quesh,Quesh Date,sent_ans,sent_quesh
0,"Do you sleep on your back, side or belly?",2021-09-19,0.000000,0.0
1,What is something you could never get enough of?,2021-09-20,1.875000,0.0
2,Where is somewhere you want to go next summer?,2021-09-21,2.428571,1.0


In [8]:
df3 = pd.read_csv("/content/drive/MyDrive/Data/charcount.csv")
df3.head(3)

,Quesh,Quesh Date,charcount
0,"Do you sleep on your back, side or belly?",2021-09-19,31.40
1,What is something you could never get enough of?,2021-09-20,34.25
2,Where is somewhere you want to go next summer?,2021-09-21,40.00


In [9]:
merged_df = pd.merge(df, df2, on='Quesh', how='left')
merged_df.head(3)

,Quesh,Quesh Date_x,Total Respo,Avg Rating,Quesh Date_y,sent_ans,sent_quesh
0,Which one condiment could you not live without?,2022-01-07,18,3.78,2022-01-07,0.722222,0.0
1,"If you had to win a game to save your life, wh...",2022-01-06,18,3.67,2022-01-06,0.722222,6.0
2,What’s your favorite food combo?,2022-01-05,16,3.56,2022-01-05,0.812500,2.0


In [10]:
merged_df = pd.merge(merged_df, df3, on='Quesh', how='left')
merged_df.head(3)

,Quesh,Quesh Date_x,Total Respo,Avg Rating,Quesh Date_y,sent_ans,sent_quesh,Quesh Date,charcount
0,Which one condiment could you not live without?,2022-01-07,18,3.78,2022-01-07,0.722222,0.0,2022-01-07,50.055556
1,"If you had to win a game to save your life, wh...",2022-01-06,18,3.67,2022-01-06,0.722222,6.0,2022-01-06,31.944444
2,What’s your favorite food combo?,2022-01-05,16,3.56,2022-01-05,0.812500,2.0,2022-01-05,52.250000


In [11]:
merged_df.drop(['Quesh Date_y', 'Quesh Date'], axis=1, inplace=True)
merged_df.head(3)

,Quesh,Quesh Date_x,Total Respo,Avg Rating,sent_ans,sent_quesh,charcount
0,Which one condiment could you not live without?,2022-01-07,18,3.78,0.722222,0.0,50.055556
1,"If you had to win a game to save your life, wh...",2022-01-06,18,3.67,0.722222,6.0,31.944444
2,What’s your favorite food combo?,2022-01-05,16,3.56,0.812500,2.0,52.250000


In [12]:
merged_df = merged_df.rename({'Quesh Date_x': 'Quesh Date', 'b': 'Y'}, axis=1)
merged_df.head(3)

,Quesh,Quesh Date,Total Respo,Avg Rating,sent_ans,sent_quesh,charcount
0,Which one condiment could you not live without?,2022-01-07,18,3.78,0.722222,0.0,50.055556
1,"If you had to win a game to save your life, wh...",2022-01-06,18,3.67,0.722222,6.0,31.944444
2,What’s your favorite food combo?,2022-01-05,16,3.56,0.812500,2.0,52.250000


In [13]:
merged_df.sort_values(by='Quesh Date', ascending=True, inplace=True)
merged_df.head(3)

,Quesh,Quesh Date,Total Respo,Avg Rating,sent_ans,sent_quesh,charcount
96,Which movie is the most quotable?,2021-10-03,8,4.75,-0.375000,0.0,46.625000
95,Homemade Tacos or Restaurant Tacos?,2021-10-04,10,4.60,0.300000,0.0,38.200000
94,Who has been your best teacher and why?,2021-10-05,18,4.50,5.944444,3.0,202.777778


In [14]:
merged_df.to_csv('merged_df.csv', index=False)

Kmeans


In [15]:
# merged_df.drop(['Quesh Date_y', 'Quesh Date_x', 'Quesh Date', 'charcount'], axis=1, inplace=True)


KeyError: ignored

In [16]:
merged_df.head(3)

,Quesh,Quesh Date,Total Respo,Avg Rating,sent_ans,sent_quesh,charcount
96,Which movie is the most quotable?,2021-10-03,8,4.75,-0.375000,0.0,46.625000
95,Homemade Tacos or Restaurant Tacos?,2021-10-04,10,4.60,0.300000,0.0,38.200000
94,Who has been your best teacher and why?,2021-10-05,18,4.50,5.944444,3.0,202.777778


In [17]:
merged_df = merged_df.set_index('Quesh')

In [18]:
merged_df

,Quesh Date,Total Respo,Avg Rating,sent_ans,sent_quesh,charcount
Quesh,,,,,,
Which movie is the most quotable?,2021-10-03,8,4.75,-0.375000,0.0,46.625000
Homemade Tacos or Restaurant Tacos?,2021-10-04,10,4.60,0.300000,0.0,38.200000
Who has been your best teacher and why?,2021-10-05,18,4.50,5.944444,3.0,202.777778
What weird food combination do you enjoy most?,2021-10-06,14,3.86,1.285714,0.0,47.285714
What has been the best decision you've ever made?,2021-10-07,11,4.00,1.636364,3.0,108.545455
...,...,...,...,...,...,...
Which song(s) sum up your life?,2022-01-03,13,4.23,0.461538,0.0,77.461538
Which ad jingle comes to mind first?,2022-01-04,21,3.57,1.476190,0.0,26.809524
What’s your favorite food combo?,2022-01-05,16,3.56,0.812500,2.0,52.250000


In [19]:
merged_df.columns = merged_df.columns.str.lower()
merged_df.sample(3)

,quesh date,total respo,avg rating,sent_ans,sent_quesh,charcount
Quesh,,,,,,
What song(s) can’t you help but sing along to?,2021-10-19,11,4.91,0.909091,2.0,82.636364
What are three must-haves in your dream home?,2021-11-16,15,4.33,2.066667,1.0,89.866667
What was the last concert you went to?,2021-12-06,20,3.65,2.000000,0.0,63.900000


In [20]:
merged_df.describe().T

,count,mean,std,min,25%,50%,75%,max
total respo,97.0,14.865979,4.876878,5.0000,11.000000,14.000000,18.000000,32.000000
avg rating,97.0,4.078351,0.351789,3.3000,3.820000,4.080000,4.330000,4.910000
sent_ans,97.0,1.658295,1.346482,-1.0000,0.818182,1.421053,2.153846,6.000000
sent_quesh,97.0,1.164948,1.351546,-1.0000,0.000000,1.000000,2.000000,6.000000
charcount,97.0,65.611735,29.499220,18.9375,45.052632,59.750000,80.888889,202.777778


In [21]:
KRANGE = range(2,11)
k_dict = {}
k_list = []
for i in KRANGE:
  ki = KMeans(i)
  labs = ki.fit_predict(merged_df)
  k_dict[i] = ki.inertia_ 
  k_list.append(ki.inertia_ )

ValueError: ignored

In [ ]:
sns.lineplot(KRANGE, k_list)
plt.show()

In [ ]:
k_list

In [ ]:
# fit our first kmeans cluster
k4 = KMeans(3) # initialize
k4.fit(merged_df) # fit wtih feature matrix

In [ ]:
labs = k4.predict(merged_df) # predict, output the cluster assignments
labs

In [ ]:
# how many iterations ran and inertia
print(k4.n_iter_)
print(k4.inertia_)

In [ ]:
# put these back onto the original dataset
merged_df['k4'] = labs
merged_df.sample(3)

In [ ]:
# lets do our first profile
k4_profile = merged_df.groupby('k4').mean()
k4_profile.T

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(k4_profile, cmap='Reds')
plt.show()

In [ ]:
# silo score from sklearn --- metrics module

j = merged_df.drop(columns=['k4'])
silo_overall = metrics.silhouette_score(j, k4.predict(j))
silo_overall


# silo samples
silo_samples = metrics.silhouette_samples(j, k4.predict(j))
type(silo_samples)
silo_samples.shape

# plot up the the fit
skplt.metrics.plot_silhouette(j, k4.predict(j), figsize=(7,7))
plt.show()

